In [1]:
import tensorflow as tf

2023-04-22 19:06:31.869979: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
import pickle

with open('encoded_articles.pkl','rb') as f:
    articles_dict = pickle.load(f)
articles_dict[0]['content_embed']

2023-04-22 19:06:39.389335: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


<tf.Tensor: shape=(1, 512, 768), dtype=float32, numpy=
array([[[ 2.6503448 ,  3.2016244 ,  1.3388029 , ...,  2.2058847 ,
          0.3312998 , -0.7049581 ],
        [-0.8145513 , -0.57293844, -1.6074227 , ..., -1.2925094 ,
          1.9251825 ,  0.19470358],
        [ 1.4512795 , -0.10057461, -0.5397182 , ..., -2.602287  ,
         -0.05150816,  0.66684645],
        ...,
        [ 1.6137371 ,  0.6078762 ,  1.4367299 , ...,  0.5003481 ,
          1.8043325 , -1.1023154 ],
        [ 0.48527208,  0.02323312, -1.3962259 , ...,  0.01635098,
          1.1910217 , -0.7607111 ],
        [-1.0391499 , -0.6451527 ,  0.4608687 , ...,  0.95216405,
          0.7718486 ,  0.71542233]]], dtype=float32)>

In [23]:
import numpy as np

for art in articles_dict:
    content_emb.append(art['content_embed'])
    

In [27]:
articles_dict[0]['content_embed']

<tf.Tensor: shape=(1, 512, 768), dtype=float32, numpy=
array([[[ 2.6503448 ,  3.2016244 ,  1.3388029 , ...,  2.2058847 ,
          0.3312998 , -0.7049581 ],
        [-0.8145513 , -0.57293844, -1.6074227 , ..., -1.2925094 ,
          1.9251825 ,  0.19470358],
        [ 1.4512795 , -0.10057461, -0.5397182 , ..., -2.602287  ,
         -0.05150816,  0.66684645],
        ...,
        [ 1.6137371 ,  0.6078762 ,  1.4367299 , ...,  0.5003481 ,
          1.8043325 , -1.1023154 ],
        [ 0.48527208,  0.02323312, -1.3962259 , ...,  0.01635098,
          1.1910217 , -0.7607111 ],
        [-1.0391499 , -0.6451527 ,  0.4608687 , ...,  0.95216405,
          0.7718486 ,  0.71542233]]], dtype=float32)>

In [35]:
content_emb = []
batch_size = 11
for i in range(22//11):
    batch_start = i*batch_size
    batch_end = (i+1)*batch_size
    content_emb.append([articles_dict[j]['content_embed'] for j in range(batch_start,batch_end)])

batch1 = tf.convert_to_tensor(np.stack(arrays=(np.concatenate(content_emb[0]),np.concatenate(content_emb[1]))))
batch1.shape


TensorShape([2, 11, 512, 768])

In [36]:
class EncoderBlock(tf.keras.layers.Layer):
    def __init__(self, emb_sz, num_heads, key_dim, **kwargs):
        super(EncoderBlock, self).__init__(**kwargs)

        # dense layers for encoder block --> NOTE TO GENERALIZE 
        self.ff_layer = tf.keras.Sequential([
            tf.keras.layers.Dense(128, activation="relu"),
            tf.keras.layers.Dense(768)
        ])
        
        # self attention layer 
        self.self_atten = tf.keras.layers.MultiHeadAttention(num_heads, key_dim)
        
        # normailization layers 
        self.layer_norm_1 =  tf.keras.layers.LayerNormalization()
        self.layer_norm_2 =  tf.keras.layers.LayerNormalization()
       

#     @tf.function
    def call(self, embedded_articles):
        
        '''
        embedded_artiles: (batch_size (TBD), window_size (512), embedding_size (768))
        '''
        # self attention on embedded articles --> z (window_size x key_dims)
        z_matrix = self.self_atten(embedded_articles, embedded_articles)
        
        # add part of Add and Normalize 
        residuals = embedded_articles + z_matrix
        
        # normalize the added matrixes 
        normalized_resid = self.layer_norm_1(residuals)
        
        # feed forward the normalized output 
        ff_output = self.ff_layer(normalized_resid)
        
        # normalize the first normalization and the output of feed forward
        normalized_resid2 = normalized_resid + ff_output
        
        encoder_output = self.layer_norm_2(normalized_resid2)
        
#         outputs = tf.nn.relu(normalized_resid3)
        return encoder_output

In [37]:
# encoder block: embed_size=768, num_heads=4, key_dim=64)
encblock = EncoderBlock(768, 1, 64)
encblock(batch1)

<tf.Tensor: shape=(2, 11, 512, 768), dtype=float32, numpy=
array([[[[ 1.0007707 ,  1.9288177 ,  0.8544002 , ...,  1.246929  ,
          -0.600557  , -0.10351774],
         [-0.48076025, -0.02717685, -0.70863223, ..., -0.68551445,
           0.521741  ,  0.0805061 ],
         [ 0.5730688 ,  0.29923183,  0.08056142, ..., -0.63599205,
          -0.43481642,  0.38445443],
         ...,
         [ 1.4230428 ,  0.51090294,  1.1639664 , ...,  1.0165987 ,
           1.0949777 , -1.0507388 ],
         [ 0.7210073 ,  0.02415081, -1.0135348 , ..., -0.36984262,
           0.96290404, -1.4017109 ],
         [-1.184756  , -0.2101558 ,  0.83744335, ...,  0.85527194,
           0.22144598, -0.18578431]],

        [[ 1.2817701 ,  1.5769377 ,  0.7427168 , ...,  1.0008177 ,
          -0.5426979 , -0.25791028],
         [-0.8403585 , -0.45353368, -0.15228042, ...,  1.6866863 ,
          -2.1070573 , -0.1213396 ],
         [ 0.48486283, -1.8812795 , -0.80471414, ...,  1.9770449 ,
          -0.28466183,  0.

In [5]:
# self attention on embedded articles --> z (window_size x key_dims)
z_matrix = self_atten(embedded_articles, embedded_articles)



NameError: name 'self_atten' is not defined